In [ ]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor
import lightgbm as lgb
import numpy as np
import pandas as pd
import warnings
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.exceptions import ConvergenceWarning  # Importing ConvergenceWarning
import os

In [ ]:
# Use correct relative paths for the CSV files
df_xbox = "Non Imputed Career Datasets/Career_Stats_Xbox.csv"
df_ps4 = "Non Imputed Career Datasets/Career_Stats_PS4.csv"
df_pc = "Non Imputed Career Datasets/Career_Stats_PC.csv"
# Read the CSV files
df_xbox = pd.read_csv(df_xbox)
df_ps4 = pd.read_csv(df_ps4)
df_pc = pd.read_csv(df_pc)


In [ ]:
os.makedirs(output_dir, exist_ok=True)
# Instead of combining, process each dataset individually
datasets = {
    "Xbox": df_xbox,
    "PS4": df_ps4,
    "PC": df_pc
}

imputed_dataframes = {}

for name, df in datasets.items():
    print(f"Processing {name} dataset...")  # Debug print
    # Convert relevant columns to numeric
    df['career_kills'] = pd.to_numeric(df['career_kills'], errors='coerce')
    df['career_wins'] = pd.to_numeric(df['career_wins'], errors='coerce')
    df['career_revives'] = pd.to_numeric(df['career_revives'], errors='coerce')

    # Replace 0s with NaN to treat them as missing values
    df.replace(0, np.nan, inplace=True)

    # Drop the player_name column for imputation
    player_names = df['player_name']
    numeric_df = df.drop(columns=['player_name'])

    # Mode Imputation
    mode_imputer = SimpleImputer(strategy="most_frequent")
    df_mode_imputed = pd.DataFrame(mode_imputer.fit_transform(numeric_df), columns=numeric_df.columns)
    df_mode_imputed.insert(0, 'player_name', player_names)
    df_mode_imputed.iloc[:, 1:] = df_mode_imputed.iloc[:, 1:].round(0).astype(int)

    # Logistic Regression Imputation
    lr_imputer = IterativeImputer(estimator=LogisticRegression(solver='lbfgs'), random_state=42, max_iter=10, verbose=0)
    df_lr_imputed = pd.DataFrame(lr_imputer.fit_transform(numeric_df), columns=numeric_df.columns)
    df_lr_imputed.insert(0, 'player_name', player_names)
    df_lr_imputed.iloc[:, 1:] = df_lr_imputed.iloc[:, 1:].round(0).astype(int)

    # Random Forest Imputation
    rf_imputer = IterativeImputer(estimator=RandomForestClassifier(n_estimators=10), random_state=42, max_iter=10, verbose=0)
    df_rf_imputed = pd.DataFrame(rf_imputer.fit_transform(numeric_df), columns=numeric_df.columns)
    df_rf_imputed.insert(0, 'player_name', player_names)
    df_rf_imputed.iloc[:, 1:] = df_rf_imputed.iloc[:, 1:].round(0).astype(int)

    # KNN Imputation
    knn_imputer = KNNImputer(n_neighbors=5)
    df_knn_imputed = pd.DataFrame(knn_imputer.fit_transform(numeric_df), columns=numeric_df.columns)
    df_knn_imputed.insert(0, 'player_name', player_names)
    df_knn_imputed.iloc[:, 1:] = df_knn_imputed.iloc[:, 1:].round(0).astype(int)

    # Mean Imputation
    mean_imputer = SimpleImputer(strategy='mean')
    df_mean_imputed = pd.DataFrame(mean_imputer.fit_transform(numeric_df), columns=numeric_df.columns)
    df_mean_imputed.insert(0, 'player_name', player_names)
    df_mean_imputed.iloc[:, 1:] = df_mean_imputed.iloc[:, 1:].round(0).astype(int)

    # Median Imputation
    median_imputer = SimpleImputer(strategy='median')
    df_median_imputed = pd.DataFrame(median_imputer.fit_transform(numeric_df), columns=numeric_df.columns)
    df_median_imputed.insert(0, 'player_name', player_names)
    df_median_imputed.iloc[:, 1:] = df_median_imputed.iloc[:, 1:].round(0).astype(int)

    # XGBoost Imputation
    xgb_imputer = IterativeImputer(estimator=XGBRegressor(), random_state=42, max_iter=10, verbose=0)
    df_xgb_imputed = pd.DataFrame(xgb_imputer.fit_transform(numeric_df), columns=numeric_df.columns)
    df_xgb_imputed.insert(0, 'player_name', player_names)
    df_xgb_imputed.iloc[:, 1:] = df_xgb_imputed.iloc[:, 1:].round(0).astype(int)

    # MICE Imputation
    mice_imputer = IterativeImputer(max_iter=10, random_state=42)
    df_mice_imputed = pd.DataFrame(mice_imputer.fit_transform(numeric_df), columns=numeric_df.columns)
    df_mice_imputed.insert(0, 'player_name', player_names)
    df_mice_imputed.iloc[:, 1:] = df_mice_imputed.iloc[:, 1:].round(0).astype(int)

    # Store all rounded imputed dataframes for this dataset
    imputed_dataframes[name] = {
        "Mode Imputation": df_mode_imputed,
        "Logistic Regression Imputation": df_lr_imputed,
        "Random Forest Imputation": df_rf_imputed,
        "KNN Imputation": df_knn_imputed,
        "Mean Imputation": df_mean_imputed,
        "Median Imputation": df_median_imputed,
        "XGBoost Imputation": df_xgb_imputed,
        "MICE Imputation": df_mice_imputed
    }

    # Only save the MICE-imputed dataframe for this dataset to the specified directory
    mice_csv_path = os.path.join(output_dir, f"MICE_Imputed_{name}.csv")
    try:
        df_mice_imputed.to_csv(mice_csv_path, index=False)
        print(f"MICE-imputed dataset for {name} saved to {mice_csv_path}")
    except Exception as e:
        print(f"Failed to save {name}: {e}")

In [ ]:
# Check missing values for each original dataset
for name, df in datasets.items():
    print(f"\n{name} - Missing entries per column:")
    print(df.isnull().sum())
    total_missing = df.isnull().sum().sum()
    print(f"{name} - Total missing entries: {total_missing}")

# Check that there are no missing values in each imputed dataframe for each dataset
for dataset_name, imputations in imputed_dataframes.items():
    print(f"\nChecking imputed dataframes for {dataset_name}:")
    for method_name, imputed_df in imputations.items():
        missing = imputed_df.isnull().sum().sum()
        print(f"  {method_name}: {missing} missing values")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_correlation_heatmap(df, title):
    plt.figure(figsize=(8, 6))
    sns.heatmap(df.corr(), annot=True, cmap="coolwarm", fmt='.2f')
    plt.title(title)
    plt.show()

for name, df in datasets.items():
    print(f"\n{name} - Correlation Heatmap BEFORE MICE Imputation:")
    numeric_df = df.drop(columns=['player_name'])
    plot_correlation_heatmap(numeric_df, f"{name} - Before MICE Imputation")

    print(f"\n{name} - Correlation Heatmap AFTER MICE Imputation:")
    mice_df = imputed_dataframes[name]["MICE Imputation"].drop(columns=['player_name'])
    plot_correlation_heatmap(mice_df, f"{name} - After MICE Imputation")

In [ ]:
# Calculate and print mean and variance for only the MICE-imputed datasets for PC, Xbox, and PS4

for name in ["PC", "Xbox", "PS4"]:
    print(f"\n{name} - Mean and Variance (MICE Imputed):")
    mice_df = imputed_dataframes[name]["MICE Imputation"].drop(columns=['player_name'])
    mean_variance = mice_df.describe().loc[['mean', 'std']].T
    mean_variance.columns = ['Mean', 'Variance']
    print(mean_variance)
                        